In [1]:
#%pip install transformers==4.38.1
#%pip install rdkit==2023.9.4
#%pip install accelerate==0.27.2
#%pip install flash-attn
#%pip install -q -U bitsandbytes
#%pip install datasets
#%pip install loralib
#%pip install git+https://github.com/huggingface/peft.git
#%pip install sentencepiece

In [2]:
#%pip install tensorflow==2.10.0
#%pip install wandb

In [3]:
%pwd 

'/mnt/data/LLM-mol-encoder-main/notebooks'

In [4]:
import random, pickle, json, os
from datasets import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import bitsandbytes as bnb
from peft import PeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, platform, warnings
import wandb
import accelerate
import bitsandbytes as bnb
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
#transformers.utils.logging.set_verbosity_debug()
import torch

import sys
sys.path.append('../credentials/')
from HF_credentials import api_token
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

/mnt/data/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
2024-05-10 06:18:54.220174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 06:18:54.311293: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation o

In [5]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig

# Tokenizer

In [6]:
llm_tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2', use_auth_token=api_token, model_max_length=256, add_prefix_space=False)
llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_tokenizer.padding_side = "right"

/mnt/data/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/mnt/data/anaconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [7]:
# Assuming llm_tokenizer is already initialized and available
# If not, you would initialize it as shown in previous examples

# Define your chat history
chat = [
  {"role": "user", "content": ""},
  {"role": "assistant", "content": ""}
]

# Apply the chat template using the tokenizer
formatted_chat = llm_tokenizer.apply_chat_template(chat, tokenize=False)

# Now wrap this formatted chat with '<s>' and '</s>'
formatted_chat_with_tokens = f"<s>{formatted_chat}</s>"

# Print the result
print(formatted_chat_with_tokens)

<s><s>[INST]  [/INST]</s></s>


# Data

In [8]:
def create_datasets(split='train'):

    conversations = []
    input_smiles = []

    with open(f'../data/LlaSMol/{split}/property_prediction-bbbp.jsonl', 'r') as f:
        for line in f:
            txt = json.loads(line)
            chat[0]['content'] = f"Is blood-brain barrier permeability (BBBP) a property of <SMILES> {txt['input']} </SMILES>?"
            chat[1]['content'] = f"<BOOLEAN> {txt['output']} </BOOLEAN>"
            # conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=True, truncation=True, padding='max_length', max_length=256))
            conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=False))
            input_smiles.append(txt['input'])
    print(conversations[-1])

    with open(f'../data/LlaSMol/{split}/property_prediction-clintox.jsonl', 'r') as f:
        for line in f:
            txt = json.loads(line)
            chat[0]['content'] = f"Is <SMILES> {txt['input']} </SMILES> toxic?"
            chat[1]['content'] = f"<BOOLEAN> {txt['output']} </BOOLEAN>"
            # conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=True, truncation=True, padding='max_length', max_length=256))
            conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=False))
            input_smiles.append(txt['input'])
    print(conversations[-1])

    with open(f'../data/LlaSMol/{split}/property_prediction-esol.jsonl', 'r') as f:
        for line in f:
            txt = json.loads(line)
            chat[0]['content'] = f"How soluble is <SMILES> {txt['input']} </SMILES>?"
            chat[1]['content'] = f"Its log solubility is <NUMBER> {txt['output']} </NUMBER> mol/L"
            # conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=True, truncation=True, padding='max_length', max_length=256))
            conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=False))
            input_smiles.append(txt['input'])
    print(conversations[-1])

    with open(f'../data/LlaSMol/{split}/property_prediction-hiv.jsonl', 'r') as f:
        for line in f:
            txt = json.loads(line)
            chat[0]['content'] = f"Can <SMILES> {txt['input']} </SMILES> serve as an inhibitor of HIV replication?"
            chat[1]['content'] = f"<BOOLEAN> {txt['output']} </BOOLEAN>"
            # conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=True, truncation=True, padding='max_length', max_length=256))
            conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=False))
            input_smiles.append(txt['input'])
    print(conversations[-1])

    with open(f'../data/LlaSMol/{split}/property_prediction-lipo.jsonl', 'r') as f:
        for line in f:
            txt = json.loads(line)
            chat[0]['content'] = f"Predict the octanol/water distribution coefficient logD under the circumstances of pH 7.4 for <SMILES> {txt['input']} </SMILES>"
            chat[1]['content'] = f"<NUMBER> {txt['output']} </NUMBER>"
            # conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=True, truncation=True, padding='max_length', max_length=256))
            conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=False))
            input_smiles.append(txt['input'])
    print(conversations[-1])

    with open(f'../data/LlaSMol/{split}/property_prediction-sider.jsonl', 'r') as f:
        for line in f:
            txt = json.loads(line)
            chat[0]['content'] = f"Are there any known side effects of <SMILES> {txt['input']} </SMILES> affecting the heart?"
            chat[1]['content'] = f"<BOOLEAN> {txt['output']['Vascular disorders']} </BOOLEAN>"
            # conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=True, truncation=True, padding='max_length', max_length=256))
            conversations.append(llm_tokenizer.apply_chat_template(chat, tokenize=False))
            input_smiles.append(txt['input'])
    print(conversations[-1])
    print(len(conversations))

    return conversations, input_smiles

In [9]:
%pwd

'/mnt/data/LLM-mol-encoder-main/notebooks'

In [10]:
print('Train:')
train_conversations, train_input_smiles = create_datasets('train')
print('Test:')
test_conversations, test_input_smiles = create_datasets('test')

Train:
<s>[INST] Is blood-brain barrier permeability (BBBP) a property of <SMILES> CC1(C)NC(=O)C(/C=C/C2=CC=CC=C2)O1 </SMILES>? [/INST]<BOOLEAN> Yes </BOOLEAN></s>
<s>[INST] Is <SMILES> CCCCOCCOCCOCC1=CC2=C(C=C1CCC)OCO2 </SMILES> toxic? [/INST]<BOOLEAN> No </BOOLEAN></s>
<s>[INST] How soluble is <SMILES> OCC1=CC=CC=C1OC1OC(CO)C(O)C(O)C1O </SMILES>? [/INST]Its log solubility is <NUMBER> -0.85 </NUMBER> mol/L</s>
<s>[INST] Can <SMILES> COC(=O)C1C2=C(CC3C4=CC=CC=C4NC(=O)C31)C1=CC=CC=C1N2 </SMILES> serve as an inhibitor of HIV replication? [/INST]<BOOLEAN> No </BOOLEAN></s>
<s>[INST] Predict the octanol/water distribution coefficient logD under the circumstances of pH 7.4 for <SMILES> C[C@H]1O[C@@H](N2C=NC3=C(N)N=C(OCC45CC6CC(CC(C6)C4)C5)N=C32)[C@H](O)[C@@H]1O </SMILES> [/INST]<NUMBER> 3.58 </NUMBER></s>
<s>[INST] Are there any known side effects of <SMILES> CCCCCOC(=O)NC1=NC(=O)N([C@@H]2O[C@H](C)[C@@H](O)[C@H]2O)C=C1F </SMILES> affecting the heart? [/INST]<BOOLEAN> Yes </BOOLEAN></s>
4096

In [11]:
print('Train:')
train_conversations, train_input_smiles = create_datasets('train')
print('Test:')
test_conversations, test_input_smiles = create_datasets('test')

Train:
<s>[INST] Is blood-brain barrier permeability (BBBP) a property of <SMILES> CC1(C)NC(=O)C(/C=C/C2=CC=CC=C2)O1 </SMILES>? [/INST]<BOOLEAN> Yes </BOOLEAN></s>
<s>[INST] Is <SMILES> CCCCOCCOCCOCC1=CC2=C(C=C1CCC)OCO2 </SMILES> toxic? [/INST]<BOOLEAN> No </BOOLEAN></s>
<s>[INST] How soluble is <SMILES> OCC1=CC=CC=C1OC1OC(CO)C(O)C(O)C1O </SMILES>? [/INST]Its log solubility is <NUMBER> -0.85 </NUMBER> mol/L</s>
<s>[INST] Can <SMILES> COC(=O)C1C2=C(CC3C4=CC=CC=C4NC(=O)C31)C1=CC=CC=C1N2 </SMILES> serve as an inhibitor of HIV replication? [/INST]<BOOLEAN> No </BOOLEAN></s>
<s>[INST] Predict the octanol/water distribution coefficient logD under the circumstances of pH 7.4 for <SMILES> C[C@H]1O[C@@H](N2C=NC3=C(N)N=C(OCC45CC6CC(CC(C6)C4)C5)N=C32)[C@H](O)[C@@H]1O </SMILES> [/INST]<NUMBER> 3.58 </NUMBER></s>
<s>[INST] Are there any known side effects of <SMILES> CCCCCOC(=O)NC1=NC(=O)N([C@@H]2O[C@H](C)[C@@H](O)[C@H]2O)C=C1F </SMILES> affecting the heart? [/INST]<BOOLEAN> Yes </BOOLEAN></s>
4096

In [12]:
# Assuming HF_credentials and wdb_credentials are Python files with api_token variables
from HF_credentials import api_token as hf_api_token
from wdb_credentials import api_token as wdb_api_token

# Set the environment variables
os.environ['HUGGINGFACE_TOKEN'] = hf_api_token
os.environ['WANDB_API_KEY'] = wdb_api_token

# Access environment variables
huggingface_token = os.getenv('HUGGINGFACE_TOKEN')

# Load the model and tokenizer using the Hugging Face token
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", use_auth_token=huggingface_token)
model = model.to('cpu')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", use_auth_token=huggingface_token)

/mnt/data/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.02it/s]


In [13]:
# Get secrets from environment variables
secret_hf = os.getenv("HUGGINGFACE_TOKEN")
secret_wandb = os.getenv("WANDB_API_KEY")

# Initialize wandb
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

# Define model and dataset names
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "mistral_7b_guanaco"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: suneelbvs. Use `wandb login --relogin` to force relogin


In [14]:
try:
    public_model = AutoModelForCausalLM.from_pretrained("gpt2")
    print("Public model (GPT-2) loaded successfully!")
except Exception as e:
    print("Failed to load public model:", str(e))

Public model (GPT-2) loaded successfully!


/mnt/data/anaconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
import bitsandbytes as bnb
import accelerate
#accelerate config
print("bitsandbytes version:", bnb.__version__)
print("accelerate version:", accelerate.__version__)

from transformers import AutoModelForCausalLM, logging as hf_logging

# Setup logging
import logging
logging.basicConfig(level=logging.DEBUG)
hf_logging.set_verbosity_debug()

# Base model
base_model = "mistralai/Mistral-7B-Instruct-v0.2"

try:
    # Load the model without BitsAndBytesConfig and ensure it's using CPU-compatible settings
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        # Remove or adjust the quantization and dtype for CPU compatibility
        # Removed quantization_config=bnb_config,
        # Removed torch_dtype=torch.bfloat16,
        # Adjust device_map for CPU usage
        device_map='cpu',  # Explicitly setting device_map to 'cpu'
        trust_remote_code=True,
    )

    print("Model loaded successfully.")

except Exception as e:
    print(f"Error loading model: {e}")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json HTTP/1.1" 401 0
loading configuration file config.json from cache at /home/mfiazure/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.40.2",
  "use_cache": true,
  "voca

bitsandbytes version: 0.43.1
accelerate version: 0.30.0


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.97it/s]
All model checkpoint weights were used when initializing MistralForCausalLM.

All the weights of MistralForCausalLM were initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mistralai/Mistral-7B-Instruct-v0.2/resolve/main/generation_config.json HTTP/1.1" 401 0
loading configuration file generation_config.json from cache at /home/mfiazure/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



Model loaded successfully.


In [16]:
#%pip install -i https://pypi.org/simple/ bitsandbytes
#%pip install accelerate
#%pip install bitsandbytes-cuda11x
#%pip install torch==1.11.0+cpu torchvision==0.12.0+cpu torchaudio==0.11.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
#%pip install --upgrade torch torchvision torchaudio
#%pip install --upgrade transformers
#%pip install --upgrade huggingface_hub
#%pip install --upgrade accelerate

In [17]:
#%pip install --upgrade --force-reinstall accelerate
#%pip install --force-reinstall -i https://pypi.org/simple/ bitsandbytes

In [18]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mistralai/Mistral-7B-Instruct-v0.2/resolve/main/tokenizer_config.json HTTP/1.1" 401 0
loading file tokenizer.model from cache at /home/mfiazure/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/tokenizer.model
loading file tokenizer.json from cache at /home/mfiazure/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/mfiazure/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/mfiazure/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/tokenizer_config.json


(True, True)

In [19]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [20]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

PyTorch: setting up devices


In [21]:
from datasets import Dataset
import torch

def convert_to_dataset(conversations, smiles, tokenizer, max_seq_length=None):
    # Tokenize with padding and truncation
    inputs = tokenizer(conversations, padding="max_length", truncation=True, max_length=max_seq_length, return_tensors="pt")
    labels = tokenizer(smiles, padding="max_length", truncation=True, max_length=max_seq_length, return_tensors="pt")["input_ids"]
    
    # Ensure labels are tensors
    labels = torch.tensor(labels, dtype=torch.long)
    
    # Return a Dataset
    dataset = Dataset.from_dict({
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': labels
    })
    return dataset


In [22]:
from transformers import TrainingArguments, AutoModelForCausalLM, logging as hf_logging
from trl import SFTTrainer

# Setup logging
logging.basicConfig(level=logging.DEBUG)
hf_logging.set_verbosity_debug()

# Assuming you have these functions or they are defined elsewhere
train_dataset = convert_to_dataset(train_conversations, train_input_smiles, tokenizer, max_seq_length=512)
eval_dataset = convert_to_dataset(test_conversations, test_input_smiles, tokenizer, max_seq_length=512)

# Define training arguments
training_arguments = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="steps",     # Evaluation is done (and logged) every "logging_steps".
)

# Update trainer initialization with dataset_text_field
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    dataset_text_field='input_ids',  # or the name of your primary text field
    packing=False,
)


/tmp/ipykernel_62923/789478695.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
using `logging_steps` to initialize `eval_steps` to 10
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/mnt/data/anaconda3/envs/llm/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [23]:
def formatting_func(examples):
    # This is a placeholder function; adjust it as per your data's needs
    return {
        'input_ids': examples['input_ids'],
        'labels': examples['labels'],
        'attention_mask': examples['attention_mask'] if 'attention_mask' in examples else None,
    }

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_func,  # Use the formatting function
    packing=False,
)

In [24]:
print(train_dataset[0])  # To understand the first item's structure

{'input_ids': [1, 1, 733, 16289, 28793, 1691, 4242, 28733, 1473, 426, 19644, 660, 1127, 2437, 325, 8923, 9399, 28731, 264, 3321, 302, 523, 28735, 5877, 20335, 28767, 334, 9419, 28732, 28743, 28731, 28743, 28732, 28746, 28762, 28731, 28743, 28740, 28746, 4020, 28746, 28743, 28732, 16079, 28731, 28743, 28746, 28743, 28740, 1867, 28735, 5877, 20335, 23911, 733, 28748, 16289, 14626, 5759, 28762, 26925, 28767, 1770, 1867, 5759, 28762, 26925, 28767, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [25]:
trainer.train()

Currently training with a batch size of: 8
***** Running training *****
  Num examples = 40,966
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5,121
  Number of trainable parameters = 92,274,688
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
10,4.026200,3.767288
20,3.720100,3.609577
30,3.755400,3.435749


***** Running Evaluation *****
  Num examples = 5123
  Batch size = 16
***** Running Evaluation *****
  Num examples = 5123
  Batch size = 16
***** Running Evaluation *****
  Num examples = 5123
  Batch size = 16
***** Running Evaluation *****
  Num examples = 5123
  Batch size = 16


KeyboardInterrupt: 

In [ ]:
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "predict blood-brain barrier permeability (BBBP) for COC1=CC=C2OC3=CC=CC=C3C=C(N3CCN(C)CC3)C2=C1"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
prompt = "solubility for COC1=CC=C2OC3=CC=CC=C3C=C(N3CCN(C)CC3)C2=C1"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])